# LlamaParse With MongoDB

<a href="https://colab.research.google.com/github/run-llama/llama_cloud_services/blob/main/examples/parse/demo_mongodb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we provide a straightforward example of using LlamaParse with MongoDB Atlas VectorSearch.

We illustrate the process of using llama-parse to parse a PDF document, then index the document with a MongoDB vector store, and subsequently perform basic queries against this store.

This notebook is structured similarly to quick start guides, aiming to introduce users to utilizing llama-parse in conjunction with a MongoDB Atlas VectorSearch.

Status:
| Last Executed | Version | State      |
|---------------|---------|------------|
| Aug-19-2025   | 0.6.61  | Maintained |

### Installation

In [ ]:
%pip install llama-cloud-services
%pip install "llama-index-vector-stores-mongodb>=0.8.0<0.9.0" "llama-index>=0.13.0<0.14.0"

### Setup API Keys

In [ ]:
import os

os.environ[
    "LLAMA_CLOUD_API_KEY"
] = "llx-..."  # Get it from https://cloud.llamaindex.ai/api-key
os.environ[
    "OPENAI_API_KEY"
] = "sk-..."  # Get it from https://platform.openai.com/api-keys

In [ ]:
import requests
import pymongo

from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_cloud_services import LlamaParse
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-5-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

### Download Document

We will use `Attention is all you need` paper.

In [ ]:
# The URL of the file you want to download
url = "https://arxiv.org/pdf/1706.03762.pdf"
# The local path where you want to save the file
file_path = "./attention.pdf"

# Perform the HTTP request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the file in binary write mode and save the content
    with open(file_path, "wb") as file:
        file.write(response.content)
    print("Download complete.")
else:
    print("Error downloading the file.")

Download complete.


### Parse the document using `LlamaParse`.

In [ ]:
result = await LlamaParse(
    parse_mode="parse_page_with_agent",
    model="openai-gpt-4-1-mini",
    high_res_ocr=True,
    adaptive_long_table=True,
    outlined_table_extraction=True,
    output_tables_as_HTML=True,
).aparse(file_path)

documents = result.get_text_documents(split_by_page=False)

Started parsing the file under job_id 993fa45f-f4ed-4d49-9032-794b3470305a
.

In [ ]:
# Take a quick look at some of the parsed text from the document:
print(documents[0].get_content()[10000:11000])

 sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions less than i.

3.2  Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum

                                     3
---
              Scaled Dot-Product Attention    Multi-Head Attention

                                                                          Linear
              MatMul

  SoftMax                                 

### Create `MongoDBAtlasVectorSearch`.

In [ ]:
mongo_uri = "<mongodb_uri>"

mongodb_client = pymongo.MongoClient(mongo_uri)
mongodb_vector_store = MongoDBAtlasVectorSearch(mongodb_client)

mongodb_vector_store.create_vector_search_index(
    dimensions=1536, path="embedding", similarity="cosine"
)

### Create nodes.

In [ ]:
node_parser = SentenceSplitter()

nodes = node_parser.get_nodes_from_documents(documents)

### Create Index and Query Engine.

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=mongodb_vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=2)

### Test Query

In [ ]:
query = "What is BLEU score on the WMT 2014 English-to-German translation task?"

response = query_engine.query(query)
print("\n***********New LlamaParse+ Basic Query Engine***********")
print(response)


***********New LlamaParse+ Basic Query Engine***********
28.4 BLEU


In [ ]:
# Take a look at one of the source nodes from the response
print(response.source_nodes[0].get_content())

For our big models,(described on the
bottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps
(3.5 days).

5.3  Optimizer

We used the Adam optimizer [20] with β1 = 0.9, β2 = 0.98 and ϵ = 10−9. We varied the learning
rate over the course of training, according to the formula:

              lrate = d−0.5 · min(step_num−0.5, step_num · warmup_steps−1.5)       (3)
              model

This corresponds to increasing the learning rate linearly for the first warmup_steps training steps,
and decreasing it thereafter proportionally to the inverse square root of the step number. We used
warmup_steps = 4000.

5.4  Regularization

We employ three types of regularization during training:

                                        7
---
Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the
English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.

     Model                      